# Markov Chains Model:

Funciona amb Leaderboard Score 0.15, per tan simple que és, prou bé funciona!

A Markov Chain Model is Implemented and Observed:

In [234]:
import pandas as pd
import numpy as np
import datetime
from tqdm import tqdm
import random

In [262]:
#read all the data:
items = pd.read_csv("dressipi_recsys2022/item_features.csv")
candidate = pd.read_csv("dressipi_recsys2022/candidate_items.csv")
purchase =  pd.read_csv("dressipi_recsys2022/train_purchases.csv")
sessions =  pd.read_csv("dressipi_recsys2022/train_sessions.csv")
sessions_test = pd.read_csv("dressipi_recsys2022/test_final_sessions.csv")
leaderboard = pd.read_csv("dressipi_recsys2022/test_leaderboard_sessions.csv")

In [263]:
import time
purchase['date'] = pd.to_datetime(purchase['date'])
sessions['date'] = pd.to_datetime(sessions['date'])
max_day = purchase['date'].max() #max date 31/05/2021 
min_day = max_day -datetime.timedelta(31)
cond = purchase['date'] >= min_day
new_purchases = purchase[cond]
top_seller = new_purchases['item_id'].value_counts()[0:100].index.to_list()

In [305]:
max_day = sessions['date'].max() #max date 31/05/2021 
testing_date = max_day -datetime.timedelta(31)
cond_test = sessions['date'] >= testing_date
cond_train = sessions['date'] < testing_date
cond_test_purchase = purchase.date >= testing_date
cond_train_purchase = purchase.date < testing_date
sessions_train = sessions[cond_train]
sessions_test = sessions[cond_test]
purchase_train = purchase[cond_train_purchase]
purchase_test = purchase[cond_test_purchase]

**Markov Model, only counting last item before the purchase:**

In [374]:
def occurrences_purchase(sessions2):
    occurrences = {}
    occurrences_item = {}
    antique_session = sessions2.session_id.iloc[0]
    sessions_num = 0
    for i,x in tqdm(enumerate(zip(sessions2['session_id'], sessions2['item_id']))):
        session, item = x
        if(session != antique_session):
            first = sessions2.iat[i-1,1]
            second = purchase.iat[sessions_num,1]
            if first not in occurrences:
                occurrences[first] = {}
            if second not in occurrences[first]:
                occurrences[first][second] = 0
            occurrences[first][second] += 1
            antique_session = session
            sessions_num += 1
            if first not in occurrences_item:
                occurrences_item[first] = 0
            occurrences_item[first] += 1
            
    return occurrences, occurrences_item

In [375]:
occurrences_train,occurrences_item_train = occurrences_purchase(sessions_train)
occurrences_test, occurrences_item_test = occurrences_purchase(sessions_test)
occurrences, occurrences_item = occurrences_purchase(sessions)

4273700it [01:53, 37769.33it/s]
470120it [00:09, 49562.65it/s]
4743820it [00:43, 107903.17it/s]


In [376]:
def get_probabilities(occurrences2,occurrences_item2):
    for x in tqdm(occurrences2):
        for y in occurrences2[x]:
            occurrences2[x][y] = occurrences2[x][y]/occurrences_item2[x]
    sorted_occurrence = occurrences2

    for x in tqdm(occurrences2):
        occurrences2[x] = sorted(occurrences2[x].items(),key = lambda x : x[1], reverse = True)

    return occurrences2

In [377]:
probabilities_train = get_probabilities(occurrences_train,occurrences_item_train)
probabilites_test = get_probabilities(occurrences_test,occurrences_item_test)
probabilities = get_probabilities(occurrences, occurrences_item)

100%|██████████| 21000/21000 [00:00<00:00, 63049.22it/s]


In [378]:
def generate_markov_solution(probabilities2,leaderboard2):
    solution = [[]]
    antique_session = leaderboard2.session_id.iloc[0]
    for i,x in tqdm(enumerate(zip(leaderboard2['session_id'], leaderboard2['item_id']))):
        session,item = x
        seen = 0
        if(session != antique_session):
            item = leaderboard2.iat[i-1,1]
            if item in probabilities2:
                for x in probabilities2[item]:
                    solution.append([int(antique_session),int(x[0]),seen+1])
                    seen += 1
                    if(seen == 100): break
            for j in range(100-seen):
                solution.append([int(antique_session),int(top_seller[j]),seen+1])
                seen += 1
            antique_session = session

    return solution

In [464]:
solutions_test = generate_markov_solution(probabilities_train,sessions_test)
solutions = generate_markov_solution(probabilities,leaderboard)
solutions_test.pop(0)
solutions.pop(0)

470120it [00:12, 37427.10it/s]
229354it [00:04, 50708.34it/s]


[]

In [479]:
def MRR(solutions,purchase_test):
    suma = 0
    for i in tqdm(range(len(purchase_test))):
        item = purchase_test.iat[i,1]
        if(i == 81619): break
        for j in range(100*i,100*(i+1)-1,1):
            if(solutions[j][1] == item):
                suma += 1
                break
    return suma/len(purchase_test) 

In [480]:
MRR(solutions_test,purchase_test)

100%|█████████▉| 81619/81621 [00:03<00:00, 21982.55it/s]


0.14323519682434668

In [453]:
solutions_test = pd.DataFrame(solution_test, columns = ['session_id','item_id', 'rank']).reset_index(drop = True)
solutions = pd.DataFrame(solutions, columns = ['session_id','item_id', 'rank']).reset_index(drop = True)
solutions_csv = solution.to_csv
solutions.to_csv(r'solution_Markov2.csv',index = False)

In [459]:
solutions

,session_id,item_id,rank
0,26,3260,1
1,26,20541,2
2,26,21035,3
3,26,5383,4
4,26,12939,5
...,...,...,...
4999895,4439653,23543,96
4999896,4439653,6736,97
4999897,4439653,6552,98
4999898,4439653,23868,99
